In [1]:
import librosa
import matplotlib.pyplot as plt
import IPython.display as dis
import numpy as np
import os, sys
from tqdm import tqdm
import pandas as pd
import librosa.display

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
if not os.path.isdir('/content/melspectrogram_data'):
    os.makedirs('/content/melspectrogram_data')

In [4]:
!unzip /content/gdrive/MyDrive/final/melspectrogram_data_plus.zip -d /content/melspectrogram_data

Archive:  /content/gdrive/MyDrive/final/melspectrogram_data_plus.zip
  inflating: /content/melspectrogram_data/1002914.npy  
  inflating: /content/melspectrogram_data/10124.npy  
  inflating: /content/melspectrogram_data/101529890.npy  
  inflating: /content/melspectrogram_data/101545093.npy  
  inflating: /content/melspectrogram_data/1016369.npy  
  inflating: /content/melspectrogram_data/101678251.npy  
  inflating: /content/melspectrogram_data/101721111.npy  
  inflating: /content/melspectrogram_data/1017622.npy  
  inflating: /content/melspectrogram_data/1023889.npy  
  inflating: /content/melspectrogram_data/10291.npy  
  inflating: /content/melspectrogram_data/102979321.npy  
  inflating: /content/melspectrogram_data/104184.npy  
  inflating: /content/melspectrogram_data/105014.npy  
  inflating: /content/melspectrogram_data/105187.npy  
  inflating: /content/melspectrogram_data/1055513.npy  
  inflating: /content/melspectrogram_data/105699.npy  
  inflating: /content/melspectrog

In [ ]:
if not os.path.isdir('/content/mfcc_data'):
    os.makedirs('/content/mfcc_data')

In [ ]:
!unzip -o /content/gdrive/MyDrive/hindi/mfcc_data.zip -d /content/mfcc_data

In [5]:
rawdata = pd.read_csv('/content/gdrive/MyDrive/final/newtag_1118.csv')

In [ ]:
lst = os.listdir('/content/melspectrogram_data')
lst[0]

'5663983.npy'

In [7]:
cat2d = rawdata.dropna()

In [10]:
cat2d_drop = cat2d[cat2d['중분류'].apply(lambda x : True if len(x) <= 4 else False)]

In [12]:
cat2d_drop['중분류'] = cat2d_drop['중분류'].apply(str.strip)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
cat2d_drop['중분류'].unique()

array(['평온한', '희망찬', '슬픈', '외로운', '폭발적인', '긴장된', '분노', '잔잔한', '발랄한',
       '흥겨운', '따스한', '부드러운', '신나는', '긴박함', '강렬한', '청량한', '공포', '시끄러움',
       '평화로운', '몽환적인', '쓸쓸한', '공격적인', '긴'], dtype=object)

In [40]:
lst = os.listdir('/content/melspectrogram_data')
# data_x = []
data_y = []
errors = []
for i in tqdm(range(len(cat2d_drop))):
    try:
        # data_x.append(np.load('/content/melspectrogram_data/' + str(int(cat2d_drop['song_id'].iloc[i]))+'.npy'))
        data_y.append(cat2d_drop['중분류'].iloc[i])
    except:
        errors.append(str(int(cat2d_drop['song_id'].iloc[i]))+'.npy')


100%|██████████| 3025/3025 [00:00<00:00, 119944.88it/s]


In [ ]:
rawdata.iloc[111]

Unnamed: 0         149
song_id       30829905
new_tag            신나는
Name: 111, dtype: object

In [42]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(data_y)

In [44]:
(label_encoder.classes_)

array(['강렬한', '공격적인', '공포', '긴', '긴박함', '긴장된', '따스한', '몽환적인', '발랄한',
       '부드러운', '분노', '슬픈', '시끄러움', '신나는', '쓸쓸한', '외로운', '잔잔한', '청량한',
       '평온한', '평화로운', '폭발적인', '흥겨운', '희망찬'], dtype='<U4')

In [38]:
len(label_encoder.classes_)

23

In [19]:
wav_to_img_data = []
target = []

# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for idx in tqdm(range(len(data_x))):
    for i in range(20):
        if 128 * (i+1) > len(data_x[idx][0]):
            break
        wav_to_img_data.append(data_x[idx][:, 128* i : 128 * (i+1)].reshape(128,-1,1))
        target.append(data_y[idx])
wav_to_img_data = np.array(wav_to_img_data)
target = np.array(target)

100%|██████████| 2994/2994 [00:00<00:00, 36352.69it/s]


In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Input, BatchNormalization, Activation, Flatten

def VGG_model():
    
    inputs = Input(shape=(128,128,1))
    x = Conv2D(64,3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(23, activation='softmax')(x)
    
    return tf.keras.Model(inputs, x)

In [23]:
model = VGG_model()

In [24]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      640       
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 128, 128, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                             

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy',  metrics=['acc'])

In [77]:
history = model.fit(wav_to_img_data, target, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1497/1497 [==============================] - 303s 203ms/step - loss: 1.4509 - acc: 0.5005 - val_loss: 3.5209 - val_acc: 0.1770
Epoch 2/10
1497/1497 [==============================] - 302s 202ms/step - loss: 1.2953 - acc: 0.5533 - val_loss: 3.1758 - val_acc: 0.2128
Epoch 3/10
1497/1497 [==============================] - 302s 202ms/step - loss: 1.1487 - acc: 0.6021 - val_loss: 3.4478 - val_acc: 0.1762
Epoch 4/10
1497/1497 [==============================] - 302s 202ms/step - loss: 1.0156 - acc: 0.6505 - val_loss: 3.3174 - val_acc: 0.1553
Epoch 5/10
1497/1497 [==============================] - 303s 202ms/step - loss: 0.8996 - acc: 0.6918 - val_loss: 3.3953 - val_acc: 0.1815
Epoch 6/10
1497/1497 [==============================] - 302s 202ms/step - loss: 0.7951 - acc: 0.7256 - val_loss: 3.8306 - val_acc: 0.2491
Epoch 7/10
1497/1497 [==============================] - 302s 202ms/step - loss: 0.7085 - acc: 0.7600 - val_loss: 3.7893 - val_acc: 0.1993
Epoch 8/10
1497/1497 [============

### 추론

In [31]:
test_data = pd.read_csv('/content/gdrive/MyDrive/final/test_newtag_1118.csv')

test_x = []
test_y = []
for i in tqdm(range(len(test_data))):
    try:
        test_x.append(np.load('/content/melspectrogram_data/' + str(int(test_data['song_id'].iloc[i]))+'.npy'))
        test_y.append(test_data['중분류'].iloc[i])
    except:
        pass

100%|██████████| 480/480 [00:03<00:00, 145.33it/s]


In [46]:
label_encoder.classes_

array(['강렬한', '공격적인', '공포', '긴', '긴박함', '긴장된', '따스한', '몽환적인', '발랄한',
       '부드러운', '분노', '슬픈', '시끄러움', '신나는', '쓸쓸한', '외로운', '잔잔한', '청량한',
       '평온한', '평화로운', '폭발적인', '흥겨운', '희망찬'], dtype='<U4')

In [45]:
test_y = label_encoder.transform(test_y)

In [78]:
test_whole = []
for i in range(len(test_x)):
    one_sample = []
    for j in range(len(test_x[i][0])):
        if (j+1) * 128 >= len(test_x[i][0]):
            break
        one_sample.append(test_x[i][:, 128*j : 128*(1+j)].reshape(128,128,-1))
    test_whole.append(one_sample)

In [79]:
cnt = 0
correct = 0
total_result = []
co_lst = []

for i in tqdm(range(len(test_whole))):
    infer = model.predict(np.array(test_whole[i]))
    infer = np.argmax(infer, axis=1)
    count_ = np.unique(infer, return_counts=True)
    total_result.append(count_)
    result = count_[0][np.argmax(count_[1])]
    cnt += 1
    # print(count_)
    # print(result, test_y[i])
    if result == test_y[i]:
        co_lst.append(i)
        correct += 1

  0%|          | 0/480 [00:00<?, ?it/s]

3/3 [==============================] - 0s 73ms/step


  0%|          | 1/480 [00:00<01:52,  4.24it/s]

4/4 [==============================] - 0s 55ms/step


  0%|          | 2/480 [00:00<01:53,  4.20it/s]

...

100%|█████████▉| 478/480 [01:31<00:00,  5.36it/s]

3/3 [==============================] - 0s 53ms/step


100%|█████████▉| 479/480 [01:31<00:00,  5.51it/s]

3/3 [==============================] - 0s 52ms/step


100%|██████████| 480/480 [01:31<00:00,  5.22it/s]


In [80]:
# i = 0
# temp = np.argmax(model.predict(np.array(test_whole[i])) , axis=1)
# cnts = np.unique(temp, return_counts=True)
c_n_t = 0
co = 0
el = 0
for i in range(len(total_result)):
    cnts = total_result[i]
    tsum = np.sum(cnts[1])
    topval = cnts[1][(np.argmax(cnts[1]))]
    
    if test_y[i] in cnts[0][np.argsort(cnts[1])[-3:]]:
        co+=1

    # for el in cnts[0][np.argsort(cnts[1])[-5:]]:
        # print(label_encoder.classes_[el])

    # print('---'*10)
    # if (cnts[1][np.argsort(cnts[1])[-1]] - cnts[1][np.argsort(cnts[1])[-2]]) / tsum > 0.4:
    #     # print((cnts[1][np.argsort(cnts[1])[-1]] - cnts[1][np.argsort(cnts[1])[-2]]))
    #     if cnts[0][(np.argmax(cnts[1]))] == test_y[i]:
    #         co += 1
    #     # print(cnts[0][(np.argmax(cnts[1]))])
    #     # print(test_y[i])
    #     # print(cnts)
    # else:
    #     el += 1
    #     if cnts[0][np.argsort(cnts[1])[-2]] == test_y[i]:
    #         co += 1
    #     elif cnts[0][(np.argmax(cnts[1]))] == test_y[i]:
    #         co += 1
    #     elif len(cnts[0]) >= 3:
    #         if cnts[0][np.argsort(cnts[1])[-3]] == test_y[i]:
    #             co += 1
    #         elif len(cnts[0]) >= 4:
    #             if cnts[0][np.argsort(cnts[1])[-4]] == test_y[i]:
    #                 co += 1
    c_n_t += 1

In [81]:
co / c_n_t

0.4166666666666667

In [83]:
model.save('/content/gdrive/MyDrive/final/vgg_cat2_1118.h5')

In [ ]:
c_n_t - el

96

In [ ]:
el

384

In [ ]:
co/c_n_t

0.61875

In [ ]:
np.array(total_result)[co_lst]

False

5

In [ ]:
label_encoder.classes_

array(['공격적인', '긴장감', '밝은', '신나는', '우울한', '차분한'], dtype='<U4')

In [ ]:
ddddd = np.load('/content/melspectrogram_data/' + '460396'+'.npy')

In [ ]:
music = []
# for idx in tqdm(range(len(data_x))):
idx = 111
for i in range(len(data_x[idx][0])):
    if i+128 == len(data_x[idx][0]):
        break
    music.append(data_x[idx][:, i : 128 + i].reshape(128,128,-1))

In [ ]:
music = []
for i in range(len(ddddd[0])):
    if i+128 == len(ddddd[0]):
        break
    music.append(ddddd[:, i : 128 + i].reshape(128,128,-1))

In [ ]:
music = []
for i in range(len(ddddd[0])):
    if 128*(i+1) >= len(ddddd[0]):
        break
    music.append(ddddd[:, 128*i : 128 * (1+ i)].reshape(128,128,-1))

In [ ]:
result = model.predict(np.array(music))

4/4 [==============================] - 1s 257ms/step


In [ ]:
res = np.argmax(result, axis=1)

In [ ]:
np.unique(res, return_counts=True)

(array([0, 2, 3, 4, 5]), array([23, 16, 16, 27, 15]))

In [ ]:
label_encoder.classes_

array(['공격적인', '긴장감', '밝은', '신나는', '우울한', '차분한'], dtype='<U4')

In [ ]:
data_y[111]

3

In [ ]:
np.unique(rawdata['senti_tag'],return_counts=True)

(array(['불안', '슬픔', '신남', '편안해요', '행복해요', '화나요'], dtype=object),
 array([200, 200, 200, 384, 151, 202]))

In [ ]:
rawdata[620:650]

,song_id,song_name,song_lyric,senti_tag
620,31246870,한(一),Do you remember you remember\nremember what yo...,행복해요
621,3628521,4AM,일어난 건지 \n아직 잠들지 못한 건지\n당장이 내일인지 오늘인지 \n어젠지도 모르...,행복해요
622,30689468,결혼,결혼에 대하여 \n예쁜 단어를 골라 \n예쁜 칭찬을 하고\n예쁜 밤을 만들 것\n결...,행복해요
623,32183386,Blueming,‘뭐해?‘라는 두 글자에\n‘네가 보고 싶어’ 나의 속마음을 담아 우\n이모티콘 하...,행복해요
624,3772006,Love Love Echo Song,No lyric,행복해요
625,30677447,Who's Got You Singing Again,It happened so easy at the start\nGot too used...,행복해요
626,32823088,On and On,I never thought I had a chance\nYou seemed so ...,행복해요
627,30677444,Futures,Wind on the cool grey water \nVoices way back ...,행복해요
628,31048045,Me & You ◑,One small step for man \none giant leap for ma...,행복해요
629,31263803,I Might ◑,Say hello to Tokyo\nI can see myself living ri...,행복해요


In [ ]:
lst = os.listdir('/content/mfcc_data')
datam_x = []
datam_y = []
for i in tqdm(range(len(rawdata))):
   datam_x.append(np.load('/content/mfcc_data/' + str(rawdata['song_id'].iloc[i])+'.npy'))
   datam_y.append(rawdata['senti_tag'].iloc[i])

100%|██████████| 1337/1337 [00:02<00:00, 632.95it/s]


In [ ]:
datam_x[0].shape

(50, 10231)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
datam_y = label_encoder.fit_transform(datam_y)

In [ ]:
dda = []
tt = []

# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for idx in tqdm(range(len(datam_x))):
    for i in range(20):
        if 100 * (i+1) > len(datam_x[idx][0]):
            break
        dda.append(datam_x[idx][:, 100* i : 100 * (i+1)].reshape(50,100,1))
        tt.append(datam_y[idx])
dda = np.array(dda)
tt = np.array(tt)

100%|██████████| 1337/1337 [00:00<00:00, 35145.30it/s]


In [ ]:
dda[0].shape

(50, 256, 1)

In [ ]:
!nvidia-smi

Tue Nov 15 06:45:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |  14632MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Input, BatchNormalization, Activation, Flatten

def mfcc():
    
    inputs = Input(shape=(50,100,1))
    x = Conv2D(64,3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    # x = Conv2D(512,3, padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = Conv2D(512,3, padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = Conv2D(512,3, padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = MaxPooling2D(2)(x)
    
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='softmax')(x)
    
    return tf.keras.Model(inputs, x)

In [ ]:
mfcc_model = mfcc()

In [ ]:
mfcc_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 50, 100, 1)]      0         
                                                                 
 conv2d_74 (Conv2D)          (None, 50, 100, 64)       640       
                                                                 
 batch_normalization_74 (Bat  (None, 50, 100, 64)      256       
 chNormalization)                                                
                                                                 
 activation_74 (Activation)  (None, 50, 100, 64)       0         
                                                                 
 conv2d_75 (Conv2D)          (None, 50, 100, 64)       36928     
                                                                 
 batch_normalization_75 (Bat  (None, 50, 100, 64)      256       
 chNormalization)                                          

In [ ]:
mfcc_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy',  metrics=['acc'])

In [ ]:
history_ = mfcc_model.fit(dda, tt, epochs=2, batch_size=32, validation_split=0.4)

Epoch 1/2
502/502 [==============================] - 44s 89ms/step - loss: 0.1180 - acc: 0.9546 - val_loss: 3.6125 - val_acc: 0.5026
Epoch 2/2
502/502 [==============================] - 43s 85ms/step - loss: 0.1211 - acc: 0.9526 - val_loss: 4.2967 - val_acc: 0.3862


In [ ]:
mfcc_model.evaluate(dda, tt)

836/836 [==============================] - 18s 22ms/step - loss: 1.8442 - acc: 0.7092


[1.8442332744598389, 0.7091622948646545]